In [1]:
import numpy as np

In [19]:
def create_train_data():
    data = [['Sunny', 'Hot', 'High', 'Weak', 'no'],
            ['Sunny', 'Hot', 'High', 'Strong', 'no'],
            ['Overcast', 'Hot', 'High', 'Weak', 'yes'],
            ['Rain', 'Mild', 'High', 'Weak', 'yes'],
            ['Rain', 'Cool','Normal', 'Weak', 'yes'],
            ['Rain', 'Cool', 'Normal', 'Strong', 'no'],
            ['Overcast', 'Cool', 'Normal', 'Strong', 'yes'],
            ['Overcast', 'Mild', 'High', 'Weak', 'no'],
            ['Sunny', 'Cool', 'Normal', 'Weak', 'yes'],
            ['Rain', 'Mild', 'Normal', 'Weak', 'yes']]

    return np.array(data)

train_data = create_train_data()
print(train_data)

[['Sunny' 'Hot' 'High' 'Weak' 'no']
 ['Sunny' 'Hot' 'High' 'Strong' 'no']
 ['Overcast' 'Hot' 'High' 'Weak' 'yes']
 ['Rain' 'Mild' 'High' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Strong' 'no']
 ['Overcast' 'Cool' 'Normal' 'Strong' 'yes']
 ['Overcast' 'Mild' 'High' 'Weak' 'no']
 ['Sunny' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Mild' 'Normal' 'Weak' 'yes']]


In [35]:
def compute_prior_probability(train_data):
    y_unique = ['no', 'yes']
    prior_probability = np.zeros(len(y_unique))
    count = {'yes': 0, 'no': 0}
    for i in train_data[:, -1]:
        if i == 'yes':
            count['yes'] += 1
        else:
            count['no'] += 1
    prior_probability[0] = count['no']/len(train_data[:, -1])
    prior_probability[1] = count['yes']/len(train_data[:, -1])

    return prior_probability

prior_probablity = compute_prior_probability(train_data)
print ("P(play tennis = No) = " , prior_probablity[0])
print ("P(play tennis = Yes) = " , prior_probablity[1])


P(play tennis = No) =  0.4
P(play tennis = Yes) =  0.6


In [79]:
train_data.shape[1] - 1

4

In [141]:
a = np.where(train_data[:,-1]=='yes')[0]
len(a)

6

In [146]:
def compute_conditional_probability(train_data):
    y_unique = ['no', 'yes']
    conditional_probability = []
    list_x_name = []
    
    for i in range(0, train_data.shape[1] - 1):
        x_unique = np.unique(train_data[:, i])
        list_x_name.append(x_unique)
        x_conditional_probability = []
        
        for y in y_unique:
            prob_each_y = []
            for x in x_unique:
                count_x = np.where((train_data[:,i]==x) & (train_data[:,-1]==y))[0]
                count_y = np.where(train_data[:,-1]==y)[0]
                prob_each_y.append(len(count_x)/len(count_y))
            x_conditional_probability.append(prob_each_y)
        
        
        conditional_probability.append(x_conditional_probability)
        
    
    return conditional_probability, list_x_name

train_data = create_train_data()
_, list_x_name = compute_conditional_probability(train_data)
print('x1 = ', list_x_name[0])
print('x2 = ', list_x_name[1])
print('x3 = ', list_x_name[2])
print('x4 = ', list_x_name[3])

x1 =  ['Overcast' 'Rain' 'Sunny']
x2 =  ['Cool' 'Hot' 'Mild']
x3 =  ['High' 'Normal']
x4 =  ['Strong' 'Weak']


In [148]:
def get_index_from_value(feature_name, list_features):
    return np.where(list_features == feature_name)[0][0]

In [147]:
train_data = create_train_data()
prob , list_x_name = compute_conditional_probability(train_data)
outlook = list_x_name[3]

i1 = get_index_from_value("Strong", outlook )
i2 = get_index_from_value('Weak', outlook)
print(i1, i2)

0 1


In [150]:
train_data = create_train_data()
conditional_probability, list_x_name = compute_conditional_probability(train_data)
# Compute P(" Outlook "=" Sunny "| Play Tennis "=" Yes ")
x1 = get_index_from_value ("Sunny", list_x_name[0])
print(x1)
print ("P('Outlook'= 'Sunny'| 'Play Tennis' = 'Yes') = ", np.round(conditional_probability[0][1][x1], 2))


2
P('Outlook'= 'Sunny'| 'Play Tennis' = 'Yes') =  0.17


In [161]:
train_data = create_train_data()
conditional_probability, list_x_name = compute_conditional_probability(train_data)
# Compute P(" Outlook "=" Sunny "| Play Tennis "=" No ")
x1 = get_index_from_value ("Sunny", list_x_name[0])
print("P('Outlook'= 'Sunny'| 'Play Tennis'= 'No') = ", np.round(conditional_probability[0][0][x1], 2))


P('Outlook'= 'Sunny'| 'Play Tennis'= 'No') =  0.5


In [179]:
def train_naive_bayes(train_data) :
    # Step 1: Calculate Prior Probability
    y_unique = ['no', 'yes']
    prior_probability = compute_prior_probability( train_data )

    # Step 2: Calculate Conditional Probability
    conditional_probability, list_x_name = compute_conditional_probability(train_data)

    return prior_probability, conditional_probability, list_x_name

In [180]:
def prediction_play_tennis(X, list_x_name, prior_probability, conditional_probability):

    x1 = get_index_from_value(X[0], list_x_name[0])
    x2 = get_index_from_value(X[1], list_x_name[1])
    x3 = get_index_from_value(X[2], list_x_name[2])
    x4 = get_index_from_value(X[3], list_x_name[3])

    p0 = conditional_probability[0][0][x1]*conditional_probability[1][0][x2]*conditional_probability[2][0][x3]*conditional_probability[3][0][x4]*prior_probability[0]
    p1 = conditional_probability[0][1][x1]*conditional_probability[1][1][x2]*conditional_probability[2][1][x3]*conditional_probability[3][1][x4]*prior_probability[1]

    if p0 > p1 :
        y_pred =0
    else:
        y_pred =1

    return y_pred

In [181]:
X = ['Sunny', 'Cool', 'High', 'Strong']
data = create_train_data()
prior_probability, conditional_probability, list_x_name = train_naive_bayes(data)
pred = prediction_play_tennis(X, list_x_name, prior_probability, conditional_probability)
if(pred):
    print("Ad should go!")
else :
    print ("Ad should not go!")


Ad should not go!
